In [5]:
# Chapter1
# NN構築試行
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score  # モデルの評価を行うための関数
from keras import metrics
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool

# 学習データ読み込み
data_X = pd.read_csv("train.csv")
# data = data_X.dropna(how="any")
# data_X.fillna("", inplace=True)
data = data_X

pd.set_option("max_columns", 100)
# pd.set_option('max_rows', 100)
data.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
0,0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,flexible,LA,t,My place is meant for family and a few friends...,2016-07-27,t,f,NaN,2016-07-13,f,2016-07-27,33.788931,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,DC,t,This is a new listing for a lovely guest bedro...,2016-09-12,t,t,100%,2015-12-30,f,2017-03-31,38.934810,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,strict,NYC,t,We're looking forward to your stay at our apt....,2016-06-15,t,f,100%,2016-05-21,t,2017-08-13,40.695118,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,SF,t,BEST CITY VIEWS - - ROOF DECK W/ BBQ & WiFi - ...,2014-03-15,t,t,100%,2012-06-19,t,2017-09-03,37.796728,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,strict,NYC,t,Charming Apartment on the upper west side of M...,2015-08-05,t,t,100%,2015-03-25,f,2017-09-10,40.785050,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


In [6]:
from sklearn.model_selection import train_test_split
import optuna

# Chapter3
# データ前処理
feature_nums = ["accommodates","bathrooms","bedrooms","beds","latitude","longitude","number_of_reviews","review_scores_rating"]
feature_obj = ["amenities","bed_type","cancellation_policy","city","cleaning_fee","description","first_review","host_has_profile_pic","host_identity_verified","host_response_rate","host_since","instant_bookable","last_review","name","neighbourhood","property_type","room_type","thumbnail_url","zipcode"]

def pre_process(input_data_X: pd.DataFrame):
    # temp_data = input_data_X.drop("amenities", axis=1)
    temp_data = input_data_X

    for feature in feature_nums:
        if feature in temp_data:
            temp_data[feature].fillna(temp_data[feature].mean(), inplace=True)
    for feature in feature_obj:
        if feature in temp_data:
            temp_data[feature].fillna("", inplace=True)

    processed_X = temp_data
    return processed_X
train_dataset = data

y_raw = train_dataset["y"]
X_raw = pre_process(train_dataset.drop(["y"], axis=1))


In [7]:
import optuna
from sklearn.metrics import mean_squared_error

def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.2)

    # パラメータの指定
    params = {
        'iterations': trial.suggest_int('iterations', 100, 200),                         
        'depth': trial.suggest_int('depth', 4, 12),                                       
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength': trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_wait': trial.suggest_int('od_wait', 10, 50),
        'loss_function': trial.suggest_categorical("loss_function", ["RMSE", "MAE"])

    }
    model = CatBoostRegressor(**params)

    categorical_features_indices_train = np.where(X_train.dtypes != np.float)[0]
    c_train = Pool(X_train, label=y_train, cat_features=categorical_features_indices_train)  

    model.fit(c_train, verbose=False)

    c_test = Pool(X_test, cat_features=categorical_features_indices_train)
    y_proba = model.predict(c_test)

    rmse_score = np.sqrt(mean_squared_error(y_test, y_proba))
    return rmse_score

In [8]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[W 2020-10-29 18:37:08,842] Trial 0 failed because of the following error: NameError("name 'CatBoostRegressor' is not defined")
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\optuna\study.py", line 709, in _run_trial
    result = func(trial)
  File "<ipython-input-7-cbfea73b02c6>", line 18, in objective
    model = CatBoostRegressor(**params)
NameError: name 'CatBoostRegressor' is not defined


NameError: name 'CatBoostRegressor' is not defined

In [60]:
params = study.best_params

In [61]:
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool


categorical_features_indices_train = np.where(X_train.dtypes != np.float)[0]
c_train = Pool(X_train, label=y_train, cat_features=categorical_features_indices_train)  

model = CatBoostRegressor(**params
        )
model.fit(c_train)

g: 49.3s
749:	learn: 44.1634451	total: 1m 39s	remaining: 49.2s
750:	learn: 44.1614891	total: 1m 39s	remaining: 49s
751:	learn: 44.1564947	total: 1m 39s	remaining: 48.9s
752:	learn: 44.1528182	total: 1m 40s	remaining: 48.8s
753:	learn: 44.1507954	total: 1m 40s	remaining: 48.6s
754:	learn: 44.1475456	total: 1m 40s	remaining: 48.5s
755:	learn: 44.1390561	total: 1m 40s	remaining: 48.4s
756:	learn: 44.1313000	total: 1m 40s	remaining: 48.3s
757:	learn: 44.1292445	total: 1m 40s	remaining: 48.1s
758:	learn: 44.1245539	total: 1m 40s	remaining: 48s
759:	learn: 44.1212876	total: 1m 41s	remaining: 47.9s
760:	learn: 44.1190215	total: 1m 41s	remaining: 47.7s
761:	learn: 44.1165667	total: 1m 41s	remaining: 47.6s
762:	learn: 44.1112273	total: 1m 41s	remaining: 47.5s
763:	learn: 44.1089848	total: 1m 41s	remaining: 47.3s
764:	learn: 44.1043490	total: 1m 41s	remaining: 47.2s
765:	learn: 44.1018322	total: 1m 41s	remaining: 47.1s
766:	learn: 44.0975673	total: 1m 42s	remaining: 46.9s
767:	learn: 44.0955179	

In [62]:
X_data = pd.read_csv("test.csv")

X_test = pre_process(X_data)

categorical_features_indices_test = np.where(X_test.dtypes != np.float)[0]
c_test = Pool(X_test, cat_features=categorical_features_indices_test)  
y_proba = model.predict(c_test)

In [63]:
np.savetxt("C:\work\AI\workspace\\10.2_民博サービスの宿泊価格予測\\result.dat", y_proba.astype("float"), fmt="%0.10f",)

In [64]:
params

{'iterations': 1120,
 'depth': 7,
 'learning_rate': 0.10983817118320867,
 'random_strength': 18,
 'bagging_temperature': 7.905855352550045,
 'od_wait': 28,
 'loss_function': 'MAE'}